# Imports

In [86]:
import pandas as pd
from unidecode import unidecode
from datetime import datetime

# Load Data

In [87]:
df_gupy = pd.read_excel('../data/data_raw/vagas_gupy.xlsx')

df_vagas = pd.concat([df_gupy], axis = 0)
df_vagas.head()

,site_da_vaga,link_site,link_origem,data_publicacao,data_coleta,posicao,titulo_da_vaga,local,modalidade,nome_empresa,contrato,regime,pcd,beneficios,codigo_vaga,descricao,data_expiracao
0,Gupy,https://encantech.gupy.io/job/eyJqb2JJZCI6NjY5...,NaN,Publicada em: 30/01/2024,2024-01-30,Analista de Dados,Pessoa Analista de Dados ll,Porto Alegre - RS,Presencial,Lojas Renner,Efetivo,NaN,Também p/ PcD,NaN,NaN,Descrição da vagaBuscamos uma pessoa muito pro...,NaN
1,Gupy,https://reclame-aqui.gupy.io/job/eyJqb2JJZCI6N...,NaN,Publicada em: 30/01/2024,2024-01-30,Analista de Dados,Analista de Dados Pleno,Não informado,Remoto,Reclame AQUI,Efetivo,NaN,Também p/ PcD,NaN,NaN,Descrição da vagaSobre o Reclame AQUIO Reclame...,NaN
2,Gupy,https://caju.gupy.io/job/eyJqb2JJZCI6NjY2NDU3M...,NaN,Publicada em: 30/01/2024,2024-01-30,Analista de Dados,Pessoa Analista de Governança de Dados Sênior,Não informado,Remoto,Caju Benefícios,Efetivo,NaN,Também p/ PcD,NaN,NaN,Descrição da vagaA Caju é uma empresa brasilei...,NaN
3,Gupy,https://digio.gupy.io/job/eyJqb2JJZCI6NjY1NTEy...,NaN,Publicada em: 30/01/2024,2024-01-30,Analista de Dados,Analista de Dados Jr,Não informado,Remoto,Digio,Efetivo,NaN,NaN,NaN,NaN,Descrição da vagaNosso objetivo é transformar ...,NaN
4,Gupy,https://ajinomotoabr.gupy.io/job/eyJqb2JJZCI6N...,NaN,Publicada em: 30/01/2024,2024-01-30,Analista de Dados,Analista de Dados - Planejamento,São Paulo - SP,Híbrido,Ajinomoto do Brasil,Efetivo,NaN,NaN,NaN,NaN,Descrição da vagaEspecialista AminoScience da ...,NaN


# Tratamento dos Dados

In [88]:
df_vagas.dtypes

site_da_vaga        object
link_site           object
link_origem        float64
data_publicacao     object
data_coleta         object
posicao             object
titulo_da_vaga      object
local               object
modalidade          object
nome_empresa        object
contrato            object
regime             float64
pcd                 object
beneficios         float64
codigo_vaga        float64
descricao           object
data_expiracao     float64
dtype: object

In [89]:
df_vagas.head(1).T

,0
site_da_vaga,Gupy
link_site,https://encantech.gupy.io/job/eyJqb2JJZCI6NjY5...
link_origem,NaN
data_publicacao,Publicada em: 30/01/2024
data_coleta,2024-01-30
posicao,Analista de Dados
titulo_da_vaga,Pessoa Analista de Dados ll
local,Porto Alegre - RS
modalidade,Presencial
nome_empresa,Lojas Renner


In [90]:
df_vagas['data_publicacao'] = df_vagas['data_publicacao'].apply(lambda x: datetime.strptime(x.replace('Publicada em: ', '').replace('/', '-'),'%d-%m-%Y'))
df_vagas['data_coleta'] = pd.to_datetime(df_vagas['data_coleta'])
df_vagas['pcd'] = df_vagas['pcd'].apply(lambda x: 'Sim' if x == 'Também p/ PcD' else 'Não informado')

In [91]:
df_vagas.head(1).T

,0
site_da_vaga,Gupy
link_site,https://encantech.gupy.io/job/eyJqb2JJZCI6NjY5...
link_origem,NaN
data_publicacao,2024-01-30 00:00:00
data_coleta,2024-01-30 00:00:00
posicao,Analista de Dados
titulo_da_vaga,Pessoa Analista de Dados ll
local,Porto Alegre - RS
modalidade,Presencial
nome_empresa,Lojas Renner


# Features

## Senioridade

In [92]:
# matches para as senioridades
junior_matches = ['junior', ' jr'  , ' i', ' l ']
pleno_matches = ['pleno', ' pl', ' ii', ' ll ']
senior_matches = ['senior', ' sr', ' iii', ' lll ']

# removendo acentos e deixando letras minúsculas
df_vagas['titulo_da_vaga_tratado'] = df_vagas['titulo_da_vaga'].apply(lambda x: unidecode(x.lower()))

# função de faz matches com as senioridades das vagas
def busca_senioridade(titulo_vaga, match_list):
    senioridade = False
    for sub_string in match_list:
        if sub_string in titulo_vaga:
            senioridade = True
    
    return senioridade

# aplicando função
df_vagas['senioridade_junior'] = df_vagas['titulo_da_vaga_tratado'].apply(lambda titulo_vaga: busca_senioridade(titulo_vaga, junior_matches))
df_vagas['senioridade_pleno'] = df_vagas['titulo_da_vaga_tratado'].apply(lambda titulo_vaga: busca_senioridade(titulo_vaga, pleno_matches))
df_vagas['senioridade_senior'] = df_vagas['titulo_da_vaga_tratado'].apply(lambda titulo_vaga: busca_senioridade(titulo_vaga, senior_matches))

# categorizando senioridades
df_vagas['senioridade'] = df_vagas[['senioridade_junior', 
                                    'senioridade_pleno', 
                                    'senioridade_senior']].apply(lambda x:  'Pleno/Sênior' if (x['senioridade_pleno']) and (x['senioridade_senior']) else
                                                                            'Júnior/Pleno' if (x['senioridade_junior']) and (x['senioridade_pleno']) else
                                                                            'Júnior' if x['senioridade_junior'] else 
                                                                            'Pleno' if x['senioridade_pleno'] else
                                                                            'Sênior' if x['senioridade_senior'] else
                                                                            'Não informado'
                                                                            , axis = 1)

In [93]:
df_vagas.loc[df_vagas['senioridade'] == 'Não informado', 'titulo_da_vaga_tratado'].unique()

array(['pessoa analista de dados ll', 'analista de dados',
       'analista de dados com power bi',
       'analista de dados (foco em backstock)',
       'analista de dados controladoria - canoas ou lajeado/rs',
       'analista de dados - fortbras - sao paulo/sp',
       'analista de qualidade de dados ',
       'analista de dados - contas a receber',
       'analista de dados (e-commerce)',
       'analista de engenharia de dados ', 'analista de dados - crm - bi',
       'cas | analista de dados - foco em gestao de comunidade de dados',
       'analista de dados ',
       'analista de dados - geoprocessamento | manufacturing.',
       'analista de dados (remoto)', 'analista de dados | salvador',
       'analista de dados | mercado ',
       'analista de privacidade de dados (lgpd) - presencial brasilia',
       'analista de customer service - dados | ribas do rio pardo/ms',
       'logistica: analista de dados', 'analista de dados - comercial   ',
       'analista de dados - jundiai

## Localidade

In [94]:
df_vagas['cidade'] = df_vagas['local'].apply(lambda x: 'Não informado' if x == 'Não informado' else x.split(' - ')[0])
df_vagas['estado'] = df_vagas['local'].apply(lambda x: 'Não informado' if x == 'Não informado' else x.split(' - ')[1])

In [95]:
df_vagas['cidade']

0       Porto Alegre
1      Não informado
2      Não informado
3      Não informado
4          São Paulo
           ...      
314        Fortaleza
315         Salvador
316          Itajubá
317    Não informado
318        São Paulo
Name: cidade, Length: 319, dtype: object

# Save Dataframe

In [96]:
columns_selected = [
            'site_da_vaga',
            'link_site',
            'link_origem',
            'data_publicacao',
            'data_coleta',
            'posicao',
            'titulo_da_vaga',
            'senioridade',
            'cidade',
            'estado',
            'modalidade',
            'nome_empresa',
            'contrato',
            'regime',
            'pcd',
            'beneficios',
            'codigo_vaga',
            'descricao',
            'data_expiracao'
]

In [97]:
df_vagas[columns_selected].to_excel('../data/data_clean/vagas_gupy_clean.xlsx', index=False)

In [99]:
df_vagas[columns_selected].head()

,site_da_vaga,link_site,link_origem,data_publicacao,data_coleta,posicao,titulo_da_vaga,senioridade,cidade,estado,modalidade,nome_empresa,contrato,regime,pcd,beneficios,codigo_vaga,descricao,data_expiracao
0,Gupy,https://encantech.gupy.io/job/eyJqb2JJZCI6NjY5...,NaN,2024-01-30,2024-01-30,Analista de Dados,Pessoa Analista de Dados ll,Não informado,Porto Alegre,RS,Presencial,Lojas Renner,Efetivo,NaN,Sim,NaN,NaN,Descrição da vagaBuscamos uma pessoa muito pro...,NaN
1,Gupy,https://reclame-aqui.gupy.io/job/eyJqb2JJZCI6N...,NaN,2024-01-30,2024-01-30,Analista de Dados,Analista de Dados Pleno,Pleno,Não informado,Não informado,Remoto,Reclame AQUI,Efetivo,NaN,Sim,NaN,NaN,Descrição da vagaSobre o Reclame AQUIO Reclame...,NaN
2,Gupy,https://caju.gupy.io/job/eyJqb2JJZCI6NjY2NDU3M...,NaN,2024-01-30,2024-01-30,Analista de Dados,Pessoa Analista de Governança de Dados Sênior,Sênior,Não informado,Não informado,Remoto,Caju Benefícios,Efetivo,NaN,Sim,NaN,NaN,Descrição da vagaA Caju é uma empresa brasilei...,NaN
3,Gupy,https://digio.gupy.io/job/eyJqb2JJZCI6NjY1NTEy...,NaN,2024-01-30,2024-01-30,Analista de Dados,Analista de Dados Jr,Júnior,Não informado,Não informado,Remoto,Digio,Efetivo,NaN,Não informado,NaN,NaN,Descrição da vagaNosso objetivo é transformar ...,NaN
4,Gupy,https://ajinomotoabr.gupy.io/job/eyJqb2JJZCI6N...,NaN,2024-01-30,2024-01-30,Analista de Dados,Analista de Dados - Planejamento,Pleno,São Paulo,SP,Híbrido,Ajinomoto do Brasil,Efetivo,NaN,Não informado,NaN,NaN,Descrição da vagaEspecialista AminoScience da ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,Gupy,https://institutoatlantico.gupy.io/job/eyJqb2J...,NaN,2022-09-06,2024-01-30,Engenheiro de Dados,[Banco de Talentos] Pessoa Engenheira de Dados,Não informado,Fortaleza,CE,Remoto,Instituto Atlântico,Banco de Talentos,NaN,Sim,NaN,NaN,"Descrição da vagaOlá, somos um Instituto de Pe...",NaN
315,Gupy,https://cidacs.gupy.io/job/eyJqb2JJZCI6MjU5Mzk...,NaN,2022-07-13,2024-01-30,Engenheiro de Dados,Banco de Talento | Engenheiro de Dados Júnior,Júnior,Salvador,BA,Presencial,Cidacs,Banco de Talentos,NaN,Não informado,NaN,NaN,Descrição da vagaA equipe de Produção de Dados...,NaN
316,Gupy,https://polloengenharia.gupy.io/job/eyJqb2JJZC...,NaN,2021-07-30,2024-01-30,Engenheiro de Dados,Engenheiro Civil - Banco de Dados,Não informado,Itajubá,MG,Presencial,Pollo Engenharia,Pessoa Jurídica,NaN,Não informado,NaN,NaN,Descrição da vagaExiste um cálculo capaz de me...,NaN
317,Gupy,https://brivia.gupy.io/job/eyJqb2JJZCI6OTQwOTQ...,NaN,2021-06-04,2024-01-30,Engenheiro de Dados,Engenheiro(a) de Dados,Não informado,Não informado,Não informado,Remoto,Brivia,Banco de Talentos,NaN,Sim,NaN,NaN,Descrição da vagaVaga para nosso BANCO DE TALE...,NaN
